In [15]:
import pandas as pd #Using pandas to read csv data

from matplotlib import pyplot as plt #for ploting

import numpy as np

from scipy.fft import fft, fftfreq, fftshift

from scipy import signal

In [16]:
dataframe = pd.read_csv('./not_GMT.csv', delimiter=',', low_memory=False) #dataframe has type pandas.core.frame.dataframeFrame

totalLine = dataframe.shape[0]
timestampPerDay = 96
offsetCol = 4

def get_data_from_table_1(line, offsetCol, timestampPerDay, dataframe):
    data = dataframe.iloc[line, offsetCol:(offsetCol + timestampPerDay)]
    data = list(data)
    return data

def plot_full_figure(dataList):
    fig = plt.figure()
    ax = fig.add_axes([0.1, 0.1, 0.8, 0.8]) # main axes
    ax.plot(t, dataList)
    ax.set_xlabel('Time')
    #ax.set_title(columnDict[columnNumber])
    #yAx=list(range(0,11))
    #ax.set_yticks([maxValue/10*x for x in yAx])
    plt.ylim(0, 1.05 * 1000)
    ax.set_xticks([0,11,23,35,47,59,71,83,95])
    ax.set_xticklabels(['00:00','03:00','06:00','09:00','12:00','15:00','18:00','21:00','23:45'])
    plt.show()

t = range(0,96) #for time axes 

#window = signal.windows.tukey(8)
#why using tukey window: giu nguyen khoang thoi gian chinh can phan tich
#to test later
window = np.array([0.3 , 0.61126047, 1., 1., 1., 1.,  0.61126047, 0.3])

def average(lst):
    return sum(lst) / len(lst)

def plot_figure(data):
    #window_data = np.multiply(data,window)
    #print(average(window_data))
    fig = plt.figure()
    ax = fig.add_axes([0.1, 0.1, 0.8, 0.8]) # main axes
    #ax.set_xlim([xmin, xmax])
    ax.set_ylim([0, 1000])
    plt.plot(data)
    plt.show()
    
def plot_fft(data):
    data = np.multiply(data,window)
    #dataNP = np.asarray(data)
    dataFT = fft(data,32)
    dataFT = list(map(abs, dataFT))
    #print(average(dataFT[3:16]))
    fig = plt.figure()
    ax = fig.add_axes([0.1, 0.1, 0.8, 0.8]) # main axes
    #ax.set_xlim([xmin, xmax])
    ax.set_ylim([-300, 400])
    plt.plot(dataFT)
    plt.show()
    
def average_data(data):
    window_data = np.multiply(data,window)
    return average(window_data)

def average_four(data):
    data = np.multiply(data,window)
    dataFT = fft(data,32)
    dataFT = list(map(abs, dataFT))
    return average(dataFT[3:16]), max(dataFT)

def max_and_max_Index(lst):
    return max(lst), lst.index(max(lst))

In [17]:
listDataframe = []
i = 0
for line in range(0,totalLine):
    #plt.clf()
    date = dataframe.iloc[line,1]
    day = dataframe.iloc[line,2]
    
    data = get_data_from_table_1(line, offsetCol, timestampPerDay, dataframe)
    #plot_full_figure(data)
    
    for time in range(8,20):
        time_data = data[time*4:time*4 + 8] #get data in 2 hours
        #print(date, "hour" ,time + 1)
        #plot_figure(time_data)
        avg_data = average_data(time_data)
        #print("data: ", i)
        #print("average data value: ", avg_data)
        #plot_fft(time_data)
        avg_four, max_four = average_four(time_data)
        #print("average fourier value: ",avg_four, "max_four: ", max_four)
        max_data, max_data_index = max_and_max_Index(time_data) #helping information for undefined data
        #row = [avg_data, avg_four, max_four, max_data, max_data_index]
        row = time_data
        #row = list(map(np.int, row))
        try:
            row = [int(i) for i in row]
        except:
            continue
        #print(type(row[1]))
        listDataframe.append(row)
        i  = i + 1
        
        #list_data.append(add_data(time_data))
        #list_four.append(add_four(time_data))
    
df = pd.DataFrame(listDataframe) 
df.to_csv("9_20_hour_classification_pure_data.csv")